# Proposal Parser - PDF Text Extraction

Parser untuk mengekstrak data dari proposal startup mahasiswa (PDF) ke dalam format terstruktur untuk database dan analisis.

## Fitur:
- Extract teks dari PDF menggunakan pdfplumber
- Segmentasi otomatis berdasarkan section (BAB dan Sub-BAB)
- Pembersihan teks otomatis
- Output berupa dictionary/JSON siap untuk database
- Support konversi ke DataFrame untuk analisis

In [ ]:
import pdfplumber
import re
import pandas as pd
import json

## 1. Define ProposalParser Class

Class ini bertanggung jawab untuk:
- Mapping section heading di PDF ke nama kolom database
- Mengatur urutan section untuk ekstraksi teks

In [2]:
class ProposalParser:
    def __init__(self):
        # Mapping Heading di PDF ke Nama Kolom Database
        self.section_map = {
            r"1\.1\s+Latar\s+Belakang\s+Usaha": "txt_latar_belakang",
            r"2\.1\s+Noble\s+Purpose": "txt_noble_purpose",
            r"2\.2\s+Identifikasi\s+Konsumen": "txt_konsumen",
            r"2\.3\s+Produk\s+Inovatif": "txt_produk_inovatif",
            r"2\.4\s+Strategi\s+Pemasaran": "txt_strategi_pemasaran",
            r"2\.5\s+Sumber\s+Daya": "txt_sumber_daya",
            r"3\.1\s+Laporan/Proyeksi\s+Keuangan": "txt_keuangan_narrative",
            r"3\.2\s+Rencana\s+Anggaran\s+Belanja": "txt_rab_narrative"
        }
        
        # Urutan section untuk logika pengambilan teks (Start -> End)
        self.section_order = [
            "txt_latar_belakang",
            "txt_noble_purpose",
            "txt_konsumen",
            "txt_produk_inovatif",
            "txt_strategi_pemasaran",
            "txt_sumber_daya",
            "txt_keuangan_narrative",
            "txt_rab_narrative"
        ]

## 2. Method: Extract Text from PDF

Method ini mengekstrak seluruh teks mentah dari file PDF menggunakan pdfplumber.

In [3]:
def extract_text_from_pdf(self, pdf_path):
    """Mengekstrak seluruh teks mentah dari PDF."""
    full_text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    full_text += text + "\n"
    except Exception as e:
        return None, str(e)
    return full_text, None

# Tambahkan method ke class
ProposalParser.extract_text_from_pdf = extract_text_from_pdf

## 3. Method: Clean Text

Method ini membersihkan teks dari whitespace berlebih dan karakter yang tidak diinginkan.

In [4]:
def clean_text(self, text):
    """Membersihkan teks dari whitespace berlebih dan karakter aneh."""
    if not text:
        return ""
    # Menghapus line break berlebihan
    text = re.sub(r'\s+', ' ', text).strip()
    # Menghapus instruksi template (opsional, jika mahasiswa lupa menghapus)
    text = re.sub(r'Petunjuk Pengisian:.*?(?=[A-Z])', '', text, flags=re.IGNORECASE) 
    return text

# Tambahkan method ke class
ProposalParser.clean_text = clean_text

## 4. Method: Parse Proposal (Main Function)

Method utama yang:
1. Extract full text dari PDF
2. Segmentasi teks berdasarkan regex heading
3. Membersihkan dan menyimpan ke dictionary
4. Handle section yang tidak ditemukan

In [5]:
def parse_proposal(self, pdf_path, tenant_id=1):
    """Fungsi Utama: Ubah PDF jadi Dictionary Dataset."""
    
    # 1. Extract Full Text
    raw_text, error = self.extract_text_from_pdf(pdf_path)
    if error:
        return {"status": "Failed", "error": error}

    # 2. Segmentasi Teks Berdasarkan Regex Heading
    extracted_data = {
        "id_tenant": tenant_id,
        "parsing_status": "Success"
    }
    
    # Cari posisi start setiap section
    positions = {}
    for pattern, col_name in self.section_map.items():
        match = re.search(pattern, raw_text, re.IGNORECASE)
        if match:
            positions[col_name] = match.start()
        else:
            positions[col_name] = -1 # Section tidak ditemukan

    # Potong teks antar section
    sorted_sections = [k for k in self.section_order if positions[k] != -1]
    
    for i, col_name in enumerate(sorted_sections):
        start_pos = positions[col_name]
        
        # Tentukan end_pos (awal dari section berikutnya, atau akhir dokumen)
        if i < len(sorted_sections) - 1:
            next_col = sorted_sections[i+1]
            end_pos = positions[next_col]
        else:
            # Ini section terakhir (RAB), ambil sampai ketemu BAB IV atau akhir
            bab_penutup = re.search(r"BAB\s+IV", raw_text, re.IGNORECASE)
            end_pos = bab_penutup.start() if bab_penutup else len(raw_text)

        # Ambil teks mentah
        segment_raw = raw_text[start_pos:end_pos]
        
        # Bersihkan teks (Hapus judul heading itu sendiri)
        # Contoh: Hapus "1.1 Latar Belakang Usaha" dari isi teks
        header_pattern = [k for k, v in self.section_map.items() if v == col_name][0]
        segment_clean = re.sub(header_pattern, "", segment_raw, flags=re.IGNORECASE)
        
        # Simpan ke data
        extracted_data[col_name] = self.clean_text(segment_clean)

    # 3. Handling Kolom yang Kosong (Jika section tidak ditemukan)
    for col in self.section_order:
        if col not in extracted_data:
            extracted_data[col] = "" # Atau None
            extracted_data["parsing_status"] = "Partial"

    return extracted_data

# Tambahkan method ke class
ProposalParser.parse_proposal = parse_proposal

---

## 5. Contoh Penggunaan

### Inisialisasi Parser dan Simulasi

Karena tidak ada file PDF asli di environment ini, kita akan menggunakan data simulasi.

In [6]:
# Inisialisasi Parser
parser = ProposalParser()

# Simulasi file path (Ganti dengan path file PDF asli Anda)
dummy_pdf_path = "Proposal_Startup_Mahasiswa.pdf"

print("--- Memulai Ekstraksi ---")
print(f"File: {dummy_pdf_path}")
print(f"Total sections yang akan diparse: {len(parser.section_order)}")
print("\nSection yang akan diekstrak:")
for i, section in enumerate(parser.section_order, 1):
    print(f"{i}. {section}")

--- Memulai Ekstraksi ---
File: Proposal_Startup_Mahasiswa.pdf
Total sections yang akan diparse: 8

Section yang akan diekstrak:
1. txt_latar_belakang
2. txt_noble_purpose
3. txt_konsumen
4. txt_produk_inovatif
5. txt_strategi_pemasaran
6. txt_sumber_daya
7. txt_keuangan_narrative
8. txt_rab_narrative


### Contoh Output (Simulasi)

Berikut contoh hasil parsing jika berhasil:

In [7]:
# Contoh output jika berhasil (Dictionary)
result = {
    "id_segmentasi": 101,
    "id_tenant": 55,
    "txt_latar_belakang": "Masalah sampah plastik di Bandung semakin tak terkendali...",
    "txt_noble_purpose": "Menciptakan lingkungan bebas plastik...",
    "txt_konsumen": "Ibu rumah tangga usia 25-40 tahun...",
    "txt_produk_inovatif": "Bioplastik dari singkong yang larut air panas...",
    "txt_strategi_pemasaran": "Dijual melalui marketplace hijau...",
    "txt_sumber_daya": "Tim terdiri dari 3 orang lulusan Kimia...",
    "txt_keuangan_narrative": "Omzet bulan pertama 5 juta rupiah...",
    "txt_rab_narrative": "Dana 10 juta untuk beli mesin press...",
    "parsing_status": "Success"
}

# Convert ke JSON (siap masuk API/Database)
json_output = json.dumps(result, indent=4, ensure_ascii=False)
print("=== JSON Output ===")
print(json_output)

=== JSON Output ===
{
    "id_segmentasi": 101,
    "id_tenant": 55,
    "txt_latar_belakang": "Masalah sampah plastik di Bandung semakin tak terkendali...",
    "txt_noble_purpose": "Menciptakan lingkungan bebas plastik...",
    "txt_konsumen": "Ibu rumah tangga usia 25-40 tahun...",
    "txt_produk_inovatif": "Bioplastik dari singkong yang larut air panas...",
    "txt_strategi_pemasaran": "Dijual melalui marketplace hijau...",
    "txt_sumber_daya": "Tim terdiri dari 3 orang lulusan Kimia...",
    "txt_keuangan_narrative": "Omzet bulan pertama 5 juta rupiah...",
    "txt_rab_narrative": "Dana 10 juta untuk beli mesin press...",
    "parsing_status": "Success"
}


### Convert ke DataFrame

Untuk analisis data atau input ke model IndoBERT:

In [9]:
# Convert ke DataFrame (siap untuk analisis data / IndoBERT)
df = pd.DataFrame([result])

print("=== DataFrame Output ===")
print(f"Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst Row Preview:")

df.head()

=== DataFrame Output ===
Shape: (1, 11)

Columns: ['id_segmentasi', 'id_tenant', 'txt_latar_belakang', 'txt_noble_purpose', 'txt_konsumen', 'txt_produk_inovatif', 'txt_strategi_pemasaran', 'txt_sumber_daya', 'txt_keuangan_narrative', 'txt_rab_narrative', 'parsing_status']

First Row Preview:


,id_segmentasi,id_tenant,txt_latar_belakang,txt_noble_purpose,txt_konsumen,txt_produk_inovatif,txt_strategi_pemasaran,txt_sumber_daya,txt_keuangan_narrative,txt_rab_narrative,parsing_status
0,101,55,Masalah sampah plastik di Bandung semakin tak ...,Menciptakan lingkungan bebas plastik...,Ibu rumah tangga usia 25-40 tahun...,Bioplastik dari singkong yang larut air panas...,Dijual melalui marketplace hijau...,Tim terdiri dari 3 orang lulusan Kimia...,Omzet bulan pertama 5 juta rupiah...,Dana 10 juta untuk beli mesin press...,Success


---

## 6. Cara Menggunakan dengan File PDF Asli

```python
# 1. Siapkan file PDF
pdf_file = "path/to/your/Proposal_Startup.pdf"

# 2. Parse proposal
parser = ProposalParser()
result = parser.parse_proposal(pdf_file, tenant_id=1)

# 3. Cek hasil
if result.get("parsing_status") == "Success":
    print("✅ Parsing berhasil!")
    # Save to database atau export ke CSV
    df = pd.DataFrame([result])
    df.to_csv("parsed_proposal.csv", index=False)
else:
    print(f"❌ Parsing gagal: {result.get('error')}")
```

## 7. Next Steps

1. **Database Integration**: Insert `result` dictionary ke database
2. **Batch Processing**: Loop untuk parse multiple PDFs
3. **Error Handling**: Tambah validasi untuk format PDF yang berbeda
4. **IndoBERT Analysis**: Gunakan DataFrame untuk sentiment/topic analysis

In [ ]:
# 1. Siapkan file PDF
pdf_file = "dataset/Template Proposal.pdf"

# 2. Parse proposal
parser = ProposalParser()
result = parser.parse_proposal(pdf_file, tenant_id=1)

# 3. Cek hasil
if result.get("parsing_status") == "Success":
    print("Parsing berhasil!")
    df = pd.DataFrame([result])
    df.to_csv("parsed_proposal.csv", index=False)
else:
    print(f"Parsing gagal: {result.get('error')}")

✅ Parsing berhasil!


In [11]:
parsed = pd.read_csv("parsed_proposal.csv")

parsed

,id_tenant,parsing_status,txt_latar_belakang,txt_noble_purpose,txt_konsumen,txt_produk_inovatif,txt_strategi_pemasaran,txt_sumber_daya,txt_keuangan_narrative,txt_rab_narrative
0,1,Success,Jelaskan masalah utama yang melatarbelakangi t...,(Tujuan Mulia) Apa tujuan mulia dari bisnis in...,Potensial Siapa target pasar spesifik Anda? Je...,Jelaskan detail produk atau jasa Anda. Apa keu...,Bagaimana cara Anda menjangkau pelanggan? Jela...,"yang Dimiliki Jelaskan komposisi tim (hacker, ...","Jika usaha sudah berjalan, lampirkan ringkasan...",NaN
